In [ ]:
#

#
from scipy.stats import spearmanr, pearsonr
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
import seaborn as sns
import pandas as pd
import google.generativeai as genai
import time
import numpy as np
import re
from concurrent.futures import ThreadPoolExecutor
from numpy import pad
import json


In [ ]:
%run ./config.ipynb

In [ ]:
device='cpu'

In [ ]:
%run ./utils.ipynb

In [ ]:
BASEPATH = "STS_summary"

In [ ]:
df = pd.read_pickle(f'{BASEPATH}/hum_mac_summ_data.pkl')

In [ ]:
df['text_wc'] = df['text'].apply(lambda x: len(x.split()))
display(df[['text_wc', 'text']])

In [ ]:
df = df.drop(columns=['human_summaries_delocalized', 'machine_summaries_delocalized', 'text_deloc', 'text_wc'])

In [ ]:
len(df)

In [ ]:
#
df_cop = df.copy()
df_cop = df_cop.sample(frac=1, random_state=3).reset_index(drop=True)

In [ ]:

LLM_FOR_ANONYMIZATION =  ['gemini'] #['sonnet','gemini']

In [ ]:


def process_row(row, llm_used, delay, REPLACEMENT_OR_REMOVAL=None):
    row = row._asdict()  # Convert namedtuple to dictionary
    row[f'machine_summaries_anonymized_{llm_used}'] = machine_summary_anon(row, llm_used, delay, REPLACEMENT_OR_REMOVAL = REPLACEMENT_OR_REMOVAL)
    row[f'machine_summaries_anonymized_{llm_used}'] = [get_text_after_modified_text(text) for text in row[f'machine_summaries_anonymized_{llm_used}']]

    print('*** HUMAN')
    row[f'human_summaries_anonymized_{llm_used}'] = human_summary_anon(row, llm_used, delay, REPLACEMENT_OR_REMOVAL=REPLACEMENT_OR_REMOVAL)
    row[f'human_summaries_anonymized_{llm_used}'] = [get_text_after_modified_text(text) for text in row[f'human_summaries_anonymized_{llm_used}']]
    return row


for llm_used in LLM_FOR_ANONYMIZATION:
    print('llm_used', llm_used)

    for REPLACEMENT_OR_REMOVAL in ['REMOVAL']:
    
        print('REPLACEMENT_OR_REMOVAL ', REPLACEMENT_OR_REMOVAL)

        print('Anonymization started ')
        delay = 6 if llm_used == 'gemini' else 5

        num_workers = 1 #if llm_used == 'gemini' else 1

        with ThreadPoolExecutor(max_workers=num_workers) as executor:
            results = list(executor.map(lambda row: process_row(row, llm_used, delay, REPLACEMENT_OR_REMOVAL), df_cop.itertuples(index=False)))

        df_cop = pd.DataFrame(results)
        # df_cop =df_cop.head(1)
        df_cop.to_pickle(f'{BASEPATH}/hum_mac_summ_data_anonymized_LLM_used_{llm_used}_{REPLACEMENT_OR_REMOVAL}.pkl') 

        display(df_cop)

In [ ]:
display(df_cop)